# L-shaped algorithm example

We illustrate in this notebook the example 2 in Chapter 5, page 188, from Birge and Louveaux (2011), "Introduction to Stochastic Programming", 2nd edition, Springer.

In [1]:
using StochasticPrograms
using HiGHS

In [2]:
import Pkg
Pkg.update()

    Updating registry at `C:\Users\slash\.julia\registries\General.toml`
    Updating git-repo `https://github.com/JLChartrand/RDST.jl`
    Updating git-repo `https://github.com/fbastin/Jasmin.jl`
   Installed XML2_jll ─────────── v2.11.5+0
   Installed RelocatableFolders ─ v1.0.1
   Installed StaticArrays ─────── v1.6.5
   Installed Distances ────────── v0.10.10
   Installed LoggingExtras ────── v1.0.3
   Installed PDMats ───────────── v0.11.25
  No Changes to `C:\Users\slash\.julia\environments\v1.9\Project.toml`
    Updating `C:\Users\slash\.julia\environments\v1.9\Manifest.toml`
  [b4f34e82] ↑ Distances v0.10.9 ⇒ v0.10.10
  [e6f89c97] ↑ LoggingExtras v1.0.2 ⇒ v1.0.3
  [90014a1f] ↑ PDMats v0.11.19 ⇒ v0.11.25
  [05181044] ↑ RelocatableFolders v1.0.0 ⇒ v1.0.1
  [90137ffa] ↑ StaticArrays v1.6.4 ⇒ v1.6.5
  [02c8fc9c] ↑ XML2_jll v2.10.4+0 ⇒ v2.11.5+0
Precompiling project...
  ✓ RelocatableFolders
  ✓ LoggingExtras
  ✓ PDMats
  ✓ Distances
  ✓ Distances → DistancesSparseArraysExt
  ✓ XML2

In [3]:
Pkg.build("IJulia")

    Building Conda ─→ `C:\Users\slash\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\8c86e48c0db1564a1d49548d3515ced5d604c408\build.log`
    Building IJulia → `C:\Users\slash\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\47ac8cc196b81001a711f4b2c12c97372338f00c\build.log`


First, we construct the 2-stage model.

In [5]:
@stochastic_model simple_model begin
    @stage 1 begin
        @decision(simple_model, x >= 0)
        @objective(simple_model, Min, 0*x)
    end
    @stage 2 begin
        @known(simple_model, x)
        @uncertain ξ
        @recourse(simple_model, 0 <= y[i in 1:2])
        @objective(simple_model, Min, y[1] + y[2])
        @constraint(simple_model, y[1] - y[2] == ξ - x)
    end
end

Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


We now define three scenarios of equal probability.

In [6]:
ξ1 = @scenario ξ = 1 probability = 1/3
ξ2 = @scenario ξ = 2 probability = 1/3
ξ3 = @scenario ξ = 4 probability = 1/3

ξ = [ξ1, ξ2, ξ3]

3-element Vector{Scenario{NamedTuple{(:ξ,), Tuple{Int64}}}}:
 Scenario with probability 0.3333333333333333
  ξ: 1
 Scenario with probability 0.3333333333333333
  ξ: 2
 Scenario with probability 0.3333333333333333
  ξ: 4

## Deterministic equivalent

In order to have a solvable program, we must instantiate the model. By default, `StochasticPrograms.jl` will generate the derministic equivalent form.

In [7]:
sp = instantiate(simple_model, ξ, optimizer = HiGHS.Optimizer)

Stochastic program with:
 * 1 decision variable
 * 2 recourse variables
 * 3 scenarios of type Scenario
Structure: Deterministic equivalent
Solver name: HiGHS

We can check the model by printing it on the screen.

In [8]:
println(sp)

Deterministic equivalent problem
Min 0.3333333333333333 y₁[1] + 0.3333333333333333 y₁[2] + 0.3333333333333333 y₂[1] + 0.3333333333333333 y₂[2] + 0.3333333333333333 y₃[1] + 0.3333333333333333 y₃[2]
Subject to
 x in Decisions
 y₁[1] in RecourseDecisions
 y₁[2] in RecourseDecisions
 y₂[1] in RecourseDecisions
 y₂[2] in RecourseDecisions
 y₃[1] in RecourseDecisions
 y₃[2] in RecourseDecisions
 x >= 0.0
 y₁[1] >= 0.0
 y₁[2] >= 0.0
 y₂[1] >= 0.0
 y₂[2] >= 0.0
 y₃[1] >= 0.0
 y₃[2] >= 0.0
 x + y₁[1] - y₁[2] == 1.0
 x + y₂[1] - y₂[2] == 2.0
 x + y₃[1] - y₃[2] == 4.0
Solver name: HiGHS


We now solve it.

In [9]:
optimize!(sp)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(7) 0s
          3     1.0000000000e+00 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 3
Objective value     :  1.0000000000e+00
HiGHS run time      :          0.01


We can check the first stage solution with the method `value()`.

In [10]:
value(sp[1,:x])

2.0

## L-shaped

We now express the program as a 2-stage optimization problem, and explore various variants of the L-shaped decomposition algorithm.

### Multi cut

In [11]:
sp_lshaped = instantiate(simple_model, ξ, optimizer = LShaped.Optimizer)

Stochastic program with:
 * 1 decision variable
 * 2 recourse variables
 * 3 scenarios of type Scenario
Structure: Stage-decomposition
Solver name: L-shaped with disaggregate cuts

In [12]:
set_optimizer_attribute(sp_lshaped, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(sp_lshaped, SubProblemOptimizer(), HiGHS.Optimizer)

In [13]:
optimize!(sp_lshaped)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-3); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.8161932471e-01
HiGHS run time      :          0.00
Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-3); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.1816193247e+00
HiGHS run time      :          0.00
Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 

L-Shaped Gap  Time: 0:00:02 (1 iterations)
  Objective:          -Inf
  Early termination:  DUAL_INFEASIBLE
  Number of cuts:     3
  Iterations:         1


The master problem is unbounded from below as $K_1$ is not bounded.

Let modify the first-stage feasible set by imposing a large bound on $x$, here $10^9$.

In [23]:
@stochastic_model simple_model begin
    @stage 1 begin
        @decision(simple_model, 0 <= x <= 1e9)
        @objective(simple_model, Min, 0*x)
    end
    @stage 2 begin
        @known(simple_model, x)
        @uncertain ξ
        @recourse(simple_model, 0 <= y[i in 1:2])
        @objective(simple_model, Min, y[1] + y[2])
        @constraint(simple_model, y[1] - y[2] == ξ - x)
    end
end

Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


In [24]:
sp_lshaped = instantiate(simple_model, ξ, optimizer = LShaped.Optimizer)
set_optimizer_attribute(sp_lshaped, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(sp_lshaped, SubProblemOptimizer(), HiGHS.Optimizer)

println(sp_lshaped)

First-stage 
Min 0
Subject to
 x in Decisions
 x >= 0.0
 x <= 1.0e9

Second-stage 
Subproblem 1 (p = 0.33):
Min y[1] + y[2]
Subject to
 x in Known(value = 0.0)
 y[1] in RecourseDecisions
 y[2] in RecourseDecisions
 y[1] >= 0.0
 y[2] >= 0.0
 x + y[1] - y[2] == 1.0

Subproblem 2 (p = 0.33):
Min y[1] + y[2]
Subject to
 x in Known(value = 0.0)
 y[1] in RecourseDecisions
 y[2] in RecourseDecisions
 y[1] >= 0.0
 y[2] >= 0.0
 x + y[1] - y[2] == 2.0

Subproblem 3 (p = 0.33):
Min y[1] + y[2]
Subject to
 x in Known(value = 0.0)
 y[1] in RecourseDecisions
 y[2] in RecourseDecisions
 y[1] >= 0.0
 y[2] >= 0.0
 x + y[1] - y[2] == 4.0

Solver name: L-shaped with disaggregate cuts


We now solve the program.

In [25]:
optimize!(sp_lshaped)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\slash\.julia\packages\ProgressMeter\vnCY0\src\ProgressMeter.jl:626
L-Shaped Gap  Time: 0:00:00 (3 iterations)
  Objective:       1.0000000099341075
  Gap:             2.2204460269701189e-16
  Number of cuts:  7
  Iterations:      3


The first-stage solution is

In [26]:
value(sp_lshaped[1,:x])

2.000000029802323

### Single Cut

Let's check what happens when the single cut variant is used.

In [27]:
set_optimizer_attribute(sp_lshaped, Aggregator(), Aggregate())
optimize!(sp_lshaped)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\slash\.julia\packages\ProgressMeter\vnCY0\src\ProgressMeter.jl:626
L-Shaped Gap  Time: 0:00:00 (5 iterations)
  Objective:       1.0
  Gap:             0.0
  Number of cuts:  4
  Iterations:      5


More iterations are needed, to obtain a similar first-stage solution.

In [28]:
value(sp_lshaped[1,:x])

1.9999999999999996